In [1]:
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from models.detr import DETR, LossSegmentation
from models.segmentation import DETRsegm, PostProcessPanoptic
from models.matcher import HungarianMatcher
from hubconf import detr_resnet101_panoptic, detr_resnet3d_panoptic
from torchvision.transforms import Resize
from util.misc import get_world_size
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def sort_func(path):
    path_id = int(path.split('/')[-1].split('_')[1])
    return path_id

In [3]:
def show_image_and_label(image, target):
    fig, axs = plt.subplots(nrows=1,ncols=2, squeeze=False,figsize=(12, 12))
    axs[0, 0].imshow(image)
    axs[0, 0].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Image')
    axs[0, 1].imshow(target)
    axs[0, 1].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Target')

In [4]:
def uniform_downsample_image(image, down_scale = 8):
    image_shape = image.shape
    dim_0_indexes = torch.arange(0, image_shape[0], down_scale)
    dim_1_indexes = torch.arange(0, image_shape[1], down_scale)
    dim_2_indexes = torch.arange(0, image_shape[2], down_scale)
    downsampled_image = image[dim_0_indexes,:,:]
    downsampled_image = downsampled_image[:,dim_1_indexes,:]
    downsampled_image = downsampled_image[:,:,dim_2_indexes]
    return downsampled_image

In [5]:
def visualize_segmented_image(segmented_image):
#     segmented_image.squeeze()
    if segmented_image.get_device() > -1: segmented_image = segmented_image.to('cpu')
    classes = segmented_image.unique()
    colors = [[128,128,128],[255,0,0],[255,255,0],[0,255,0],[0,255,255],[0,0,255],[255,0,255]]
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for _class in classes[1:]:
        points_numpy = (segmented_image == _class).nonzero().numpy()
        o3d_point_cloud = o3d.geometry.PointCloud()
        o3d_point_cloud.points = o3d.utility.Vector3dVector(points_numpy)
        o3d_point_cloud.estimate_normals()
        o3d_point_cloud.paint_uniform_color(np.array(colors[int(_class)-1])/255)
        vis.add_geometry(o3d_point_cloud)
    vis.run()
    vis.destroy_window()

In [6]:
def get_bounding_boxes(segmented_image):
    if segmented_image.get_device() > -1: segmented_image = segmented_image.to('cpu')
    classes = segmented_image.unique()
    bb_list = []
    for class_ in classes: 
        points = (segmented_image == 1).nonzero()
        x_min, x_max = points[:,0].min(), points[:,0].max()
        y_min, y_max = points[:,1].min(), points[:,1].max()
        z_min, z_max = points[:,2].min(), points[:,2].max()
        bb = torch.tensor([(x_max-x_min)/2, (y_max-y_min)/2, (z_max-z_min)/2, x_max-x_min, y_max-y_min, z_max-z_min])
        bb[0::2] = bb[0::2]/torch.tensor(segmented_image.shape)
        bb[1::2] = bb[1::2]/torch.tensor(segmented_image.shape)
        bb_list.append(bb)
    bbs = torch.stack(bb_list)
    return bbs

In [7]:
def get_masks(segmented_image):
    if segmented_image.get_device() > -1: segmented_image = segmented_image.to('cpu')
    classes = segmented_image.unique()
    masks_list = []
    for class_ in classes:
        mask = segmented_image.clone()
        mask[segmented_image == class_] = 1
        mask[segmented_image != class_] = 0
        masks_list.append(mask.short())
    masks = torch.stack(masks_list)
    return masks

In [8]:
def get_labels(segmented_image):
    return segmented_image.unique().long()

In [9]:
def create_target_dict(segmented_image):
    labels = get_labels(segmented_image)
    boxes = get_bounding_boxes(segmented_image)
    masks = get_masks(segmented_image)
    target_dict = {'labels': labels, 'boxes': boxes, 'masks': masks}
    return target_dict

In [10]:
class DatasetForSegmentation(Dataset):
    
    def __init__(self, image_paths, target_paths, down_scale=8):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.down_scale = down_scale
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, i):
        image_np_array = nib.load(image_paths[i]).get_fdata()
        image_torch_tensor = torch.from_numpy(image_np_array)
        downsampled_image = uniform_downsample_image(image_torch_tensor, down_scale=self.down_scale)
        target_np_array = nib.load(target_paths[i]).get_fdata()
        target_torch_tensor = torch.from_numpy(target_np_array)
        downsampled_target = uniform_downsample_image(target_torch_tensor, down_scale=self.down_scale)
        target_dict = create_target_dict(downsampled_target)
        return downsampled_image, target_dict

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
path = '/home/francisco/workspace/ImageCHD_dataset'

In [13]:
image_paths = glob(f'{path}/*image.nii.gz',recursive=True)
target_paths = glob(f'{path}/*label.nii.gz',recursive=True)

In [14]:
image_paths.sort(key=sort_func)
target_paths.sort(key=sort_func)

In [15]:
dset = DatasetForSegmentation(image_paths,target_paths, down_scale=8)

In [16]:
image, target = dset[3]

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO - 2022-10-19 11:13:14,290 - batteryrunners - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO - 2022-10-19 11:13:15,394 - batteryrunners - pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


In [17]:
# image = image.half()
image = image.float()
image_reshaped = image.unsqueeze(0).unsqueeze(0).to(device)

In [18]:
detr_seg = detr_resnet3d_panoptic()
detr_seg.eval();
detr_seg.to(device);

In [19]:
outputs = detr_seg(image_reshaped)

/home/francisco/workspace/CHD_Classifier_by_Francisco_Lourenço/models/position_encoding.py:84: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


In [20]:
# for i in target.items():
#     target[i[0]] = i[1].to(device)

In [21]:
# matcher = HungarianMatcher()

In [22]:
# loss = LossSegmentation(matcher)

In [23]:
# losses = loss.loss_masks(outputs, [target])

***

# Visualization:

In [22]:
is_thing_map = {'0': False, '1': True, '2': True, '3': True, '4': True, '5': True, '6': True, '7': True}

In [23]:
post_process_panoptic = PostProcessPanoptic(is_thing_map)

In [24]:
result = post_process_panoptic(outputs,[tuple(torch.tensor(image.shape).tolist())])

In [28]:
# visualize_segmented_image(result)

tensor([0])